In [1]:
import pandas as pd

csv_file_path = 'weather_data_budapest_pestszentlorinc.csv'

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(csv_file_path, sep=';\\s*', header=0)

C:\Users\Domonkos\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [10]:
# Convert the 'Datetime' column to datetime format
df['Datetime'] = pd.to_datetime(df['Time'], format='%Y%m%d%H%M', errors='coerce')

# # Filter the DataFrame for the specified date range (May to September 2023)
# filtered_df = df[(df['Datetime'].dt.year == 2023) & (df['Datetime'].dt.month >= 5) & (df['Datetime'].dt.month <= 9)]

# print(filtered_df)

# Initialize a dictionary to store data month by month
budapest_weather_data = []

# Group the filtered DataFrame by month and iterate through each group
for name, group in df.groupby(df['Datetime'].dt.month):
    # Extract month and year for labeling
    month_year = group['Datetime'].dt.strftime('%B %Y').iloc[0]
    
    # Store the relevant data (Datetime, Outdoor drybulb temperature, Outdoor relative humidity) for each month
    month_data = {
        'Datetime': group['Datetime'],
        'Month': group['Datetime'].dt.month,
        'Day': group['Datetime'].dt.day,
        'Hour': group['Datetime'].dt.hour,
        'Outdoor Drybulb Temperature (C)': group['t'],
        'Outdoor Relative Humidity (%)': group['u']
    }

    budapest_weather_data.append(pd.DataFrame(month_data))

budapest_weather_data = pd.concat(budapest_weather_data, ignore_index=True)

display(budapest_weather_data.head())

,Datetime,Month,Day,Hour,Outdoor Drybulb Temperature (C),Outdoor Relative Humidity (%)
0,2002-01-01 00:00:00,1,1,0,-4.3,67
1,2002-01-01 01:00:00,1,1,1,-4.5,69
2,2002-01-01 02:00:00,1,1,2,-4.8,70
3,2002-01-01 03:00:00,1,1,3,-5.1,70
4,2002-01-01 04:00:00,1,1,4,-5.4,71


In [11]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

# Load historical weather data into a DataFrame (assuming it's already formatted properly)
# historical_data = ...

# Split the data into training and validation sets
train_data = budapest_weather_data.loc[:'2022-12-31']
validation_data = budapest_weather_data.loc['2023-01-01':]

# Fit ARIMA model to the training data
# Example: ARIMA model for outdoor drybulb temperature
# You may need to adjust the order and seasonal_order based on your data
model = ARIMA(train_data['Outdoor Drybulb Temperature (C)'], order=(5,1,0))
fit_model = model.fit()

# Forecast future values for the validation set
forecast_6h = fit_model.forecast(steps=6)
forecast_12h = fit_model.forecast(steps=12)
forecast_24h = fit_model.forecast(steps=24)

# You can similarly forecast for outdoor relative humidity
# Example: ARIMA model for outdoor relative humidity
# model_rh = ARIMA(train_data['relative_humidity'], order=(p,d,q))
# fit_model_rh = model_rh.fit()
# forecast_6h_rh = fit_model_rh.forecast(steps=6)
# forecast_12h_rh = fit_model_rh.forecast(steps=12)
# forecast_24h_rh = fit_model_rh.forecast(steps=24)

# Output the forecasts
print("6-hour forecast for drybulb temperature:", forecast_6h)
print("12-hour forecast for drybulb temperature:", forecast_12h)
print("24-hour forecast for drybulb temperature:", forecast_24h)

6-hour forecast for drybulb temperature: 20220    6.272142
20221    6.504005
20222    6.543655
20223    6.410675
20224    6.233165
20225    6.078831
Name: predicted_mean, dtype: float64
12-hour forecast for drybulb temperature: 20220    6.272142
20221    6.504005
20222    6.543655
20223    6.410675
20224    6.233165
20225    6.078831
20226    5.967266
20227    5.899390
20228    5.874341
20229    5.876937
20230    5.893085
20231    5.913086
Name: predicted_mean, dtype: float64
24-hour forecast for drybulb temperature: 20220    6.272142
20221    6.504005
20222    6.543655
20223    6.410675
20224    6.233165
20225    6.078831
20226    5.967266
20227    5.899390
20228    5.874341
20229    5.876937
20230    5.893085
20231    5.913086
20232    5.931057
20233    5.943642
20234    5.950569
20235    5.952976
20236    5.952421
20237    5.950373
20238    5.948000
20239    5.945983
20240    5.944602
20241    5.943868
20242    5.943648
20243    5.943750
Name: predicted_mean, dtype: float64


In [29]:
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
import numpy as np

# Assuming you have the historical data loaded into a DataFrame named historical_data
# historical_data = ...

# Subset the historical data for the summer of 2023
summer_2023 = budapest_weather_data[(budapest_weather_data['Datetime'].dt.year == 2023) & (budapest_weather_data['Datetime'].dt.month >= 6) & (budapest_weather_data['Datetime'].dt.month <= 8)]

hourly_forecasts = pd.DataFrame(columns=['timestamp', 'forecast_6h', 'forecast_12h', 'forecast_24h'])

past_years_data = budapest_weather_data[(budapest_weather_data['Datetime'].dt.year < 2022) & (budapest_weather_data['Datetime'].dt.year >= 2019)]
# past_years_data = budapest_weather_data.loc[budapest_weather_data['Datetime'].dt.year <= 2022 & budapest_weather_data['Datetime'].dt.year > 2019, 'Outdoor Drybulb Temperature (C)']

display(past_years_data)
print(np.asarray(past_years_data['Outdoor Drybulb Temperature (C)']))
model = SARIMAX(np.asarray(past_years_data['Outdoor Drybulb Temperature (C)']), order=(5, 1, 0), seasonal_order=(1, 1, 1, 24))
fit_model = model.fit()
# Loop through each hour in the summer of 2023
for timestamp in summer_2023['Datetime']:
    print(timestamp)
    # Fit SARIMA model to the historical data up to the current timestamp
    # model = SARIMAX(summer_2023.loc[:timestamp, 'Outdoor Drybulb Temperature (C)'], order=(5, 1, 0), seasonal_order=(1, 1, 1, 24))
    
    # Forecast the drybulb temperature for 6, 12, and 24 hours ahead
    forecast_6h = fit_model.forecast(steps=6)[-1]  # Forecast for 6 hours ahead
    forecast_12h = fit_model.forecast(steps=12)[-1]  # Forecast for 12 hours ahead
    forecast_24h = fit_model.forecast(steps=24)[-1]  # Forecast for 24 hours ahead
    
    # Append the forecasts to the DataFrame
    hourly_forecasts = hourly_forecasts.append({'timestamp': timestamp, 'forecast_6h': forecast_6h,
                                                'forecast_12h': forecast_12h, 'forecast_24h': forecast_24h},
                                               ignore_index=True)

# Output the hourly forecasts
print(hourly_forecasts)

,Datetime,Month,Day,Hour,Outdoor Drybulb Temperature (C),Outdoor Relative Humidity (%)
12648,2019-01-01 00:00:00,1,1,0,0.8,77
12649,2019-01-01 01:00:00,1,1,1,-0.2,80
12650,2019-01-01 02:00:00,1,1,2,0.1,81
12651,2019-01-01 03:00:00,1,1,3,-1.8,88
12652,2019-01-01 04:00:00,1,1,4,-1.7,90
...,...,...,...,...,...,...
191347,2021-12-31 19:00:00,12,31,19,9.0,87
191348,2021-12-31 20:00:00,12,31,20,8.7,88
191349,2021-12-31 21:00:00,12,31,21,9.3,86
191350,2021-12-31 22:00:00,12,31,22,8.2,90


[ 0.8 -0.2  0.1 ...  9.3  8.2  7.7]
